In [9]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [3]:
def fetch_and_parse_xml(url):
    # Fetch the content from URL
    response = requests.get(url)
    response.raise_for_status()  # This will raise an error if the fetch fails

    # Parse XML
    root = ET.fromstring(response.content)
    return root

def extract_information(root):
    # Define the namespace mapping
    namespaces = {
        'sa': 'https://seekingalpha.com/api/1.0',
        'media': 'http://search.yahoo.com/mrss/'
    }

    # Loop through each item in the XML
    articles = []
    for item in root.findall('.//channel/item', namespaces):
        # Extract information from each item
        title = item.find('title').text
        link = item.find('link').text
        pub_date = item.find('pubDate').text
        author_name = item.find('sa:author_name', namespaces).text
        
        # Collect article information in a dictionary
        article_info = {
            'title': title,
            'link': link,
            'published_date': pub_date,
            'author_name': author_name
        }
        articles.append(article_info)

    return articles

In [5]:
# URL of the RSS feed
url = "https://seekingalpha.com/api/sa/combined/AAPL.xml"

# Process the URL
root = fetch_and_parse_xml(url)

# Extract and print article information
articles = extract_information(root)
for article in articles:
    print(article)

{'title': 'Apple: Shiny No More - Sell Now And Keep Your Profits Thus Far', 'link': 'https://seekingalpha.com/article/4687585-apple-shiny-no-more-sell-now-and-keep-your-profits-thus-far?source=feed_symbol_AAPL', 'published_date': 'Tue, 30 Apr 2024 14:00:41 -0400', 'author_name': 'DeVas Research'}
{'title': 'Apple loses No.1 spot in global smartphone market to Samsung in Q1 2024', 'link': 'https://seekingalpha.com/symbol/AAPL/news?source=feed_symbol_AAPL', 'published_date': 'Tue, 30 Apr 2024 11:04:11 -0400', 'author_name': 'Vansh Agarwal'}
{'title': "Huawei's Q1 profit soars amid rising challenge to Apple in China - report", 'link': 'https://seekingalpha.com/symbol/AAPL/news?source=feed_symbol_AAPL', 'published_date': 'Tue, 30 Apr 2024 10:01:09 -0400', 'author_name': 'Ravikash Bakolia'}
{'title': 'Apple quietly builds AI dream team, but final product remains mystery: report', 'link': 'https://seekingalpha.com/symbol/AAPL/news?source=feed_symbol_AAPL', 'published_date': 'Tue, 30 Apr 2024

In [7]:
for article in articles:
    print(article['title'])

Apple: Shiny No More - Sell Now And Keep Your Profits Thus Far
Apple loses No.1 spot in global smartphone market to Samsung in Q1 2024
Huawei's Q1 profit soars amid rising challenge to Apple in China - report
Apple quietly builds AI dream team, but final product remains mystery: report
Apple's supplier list indicates China remains key manufacturing base: BofA
Apple upgraded at Bernstein ahead of earnings
Apple's iPad gets added to products under EU's Digital Markets Act
Earnings week ahead: Amazon, Apple, Pfizer, AMD, Coca-Cola, Starbucks and more
Apple faces scrutiny over iPhone sales, AI strategy in upcoming earnings call
Apple, OpenAI restart talks to add generative AI to next iPhone: Bloomberg
FOMC Meeting, Jobs Report, And Apple Earnings Headline Big Week
Apple's removal of 'nonconsensual nude' apps is latest AI battle for big tech
Huawei's new phone features updated version of advanced chip made in China - report
Apple gets queries from Congo on concerns related to use of conflic

In [13]:
# Might be against ToS and doesn't work right now
# def scrape_google_news(query):
#     url = f"https://news.google.com/search?q=apple&hl=en-US&gl=US&ceid=US%3Aen"
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

#     response = requests.get(url, headers=headers)
#     soup = BeautifulSoup(response.text, 'html.parser')

#     articles = []
#     for item in soup.find_all('div', attrs={'class': 'xrnccd'}):
#         title = item.find('h3', attrs={'class': 'ipQwMb ekueJc RD0gLb'}).text if item.find('h3', attrs={'class': 'ipQwMb ekueJc RD0gLb'}) else 'No Title'
#         link = 'https://news.google.com' + item.find('a', attrs={'class': 'VDXfz'})['href'].strip('.') if item.find('a', attrs={'class': 'VDXfz'}) else 'No Link'
        
#         articles.append({
#             'title': title,
#             'link': link
#         })

#     return articles

# # Usage
# apple_news = scrape_google_news('Apple')
# for news in apple_news:
#     print(news)

In [17]:
def load_api_key(filepath):
    """Load and return the API key from a file."""
    with open(filepath, 'r') as file:
        api_key = file.read().strip()  # .strip() removes any leading/trailing whitespace
    return api_key

# Usage
api_key_path = '"C:\Users\blkeu\Desktop\api_key.txt"'
api_key = load_api_key(api_key_path)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 3-4: truncated \UXXXXXXXX escape (1637300995.py, line 8)

In [16]:
def fetch_news(api_key, query):
    url = f'https://newsapi.org/v2/everything?q=apple&apiKey='
    url += api_key
    response = requests.get(url)
    return response.json()

api_key = 'your_api_key_here'
query = 'Apple'
news_items = fetch_news(api_key, query)

for article in news_items['articles']:
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}")
    print(f"URL: {article['url']}\n")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2529751070.py, line 1)